In [ ]:

dictionary=pd.read_csv('C:/Users/LabPC/Desktop/dictionary\\dictionary.csv')
Z=get_all_file_paths ('C:\\Users\\LabPC\\Desktop\\Final process\\1')
def get_all_file_paths(directory):
    file_paths = []

    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)

    return file_paths

s=-1
for file in Z:
    
    
    df=pd.read_csv(file)
    if df['Slope7'].unique()[0]>0:
        s=s+1
        if s==0:
            a1=ALL('Mean7 Delay','e17', 'e27', 'e37', 'e47',df)
        else:
            a1=a1.append(ALL('Mean7 Delay','e17', 'e27', 'e37', 'e47',df))
        a1=a1.append(ALL('Mean7 Delayreftravel','eR17', 'eR27', 'eR37', 'eR47',df))
        
    else:
        s=s+1
        if s==0:
            a1=ALL('Mean7 Delayreftravel','eR17', 'eR27', 'eR37', 'eR47',df)
        else:
            a1=a1.append(ALL('Mean7 Delayreftravel','eR17', 'eR27', 'eR37', 'eR47',df))
a1.to_csv('1.csv')

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import math
from scipy.special import gamma
from scipy import integrate
from sklearn.preprocessing import StandardScaler
import scipy.stats












# Aggregate All Function
def ALL(Mean7_Delay,e17, e27, e37, e47,df):
    param,param1,seasonT,EseasonT,Likespring,Likesummer,Likefall,Likewinter,Like3p,param_primespring,param1_primespring,param_primesummer,param1_primesummer,param_primefall,param1_primefall,param_primewinter,param1_primewinter,Rspring,Rsummer,Rfall,Rwinter= parameters_Eseason(Mean7_Delay,e17, e27, e37, e47,df)
    Result=pd.DataFrame([[int(df['Segment ID'].unique()[0]),e17]],columns=['Segment ID','type'])
    A=dictionary.loc[dictionary['Segment ID']==int(df['Segment ID'].unique()[0])]
    Result['Length']=A['Segment Length(Kilometers)'].unique()[0]
    INS1M2W=(inverseS1_model2('winter',EseasonT,param_primewinter,param1_primewinter))
    INS1M2SM=inverseS1_model2('summer',EseasonT,param_primesummer,param1_primesummer)
    INS1M2F=inverseS1_model2('fall',EseasonT,param_primefall,param1_primefall)
    INS1M2SP=inverseS1_model2('spring',EseasonT,param_primespring,param1_primespring)
    EseasonTEX,SeasonTEX,scenario2=sce2(EseasonT,seasonT) 
    scenario2_model2=inverseS2_model2(EseasonTEX,SeasonTEX,scenario2)
    scenario2=inverseS2(EseasonTEX,SeasonTEX,scenario2)
    INS1M1=inverseS1(EseasonT,param,param1)
    LIKDF=pd.DataFrame([0])
    LIKM2W=(Likelihood_Model2('winter',EseasonT,param_primewinter,param1_primewinter))
    LIKM2SM=Likelihood_Model2('summer',EseasonT,param_primesummer,param1_primesummer)
    LIKM2F=Likelihood_Model2('fall',EseasonT,param_primefall,param1_primefall)
    LIKM2SP=Likelihood_Model2('spring',EseasonT,param_primespring,param1_primespring)
    LIKDF['Log_CGEV_M2_spring']=LIKM2SP
    LIKDF['Log_CGEV_M2_summer']=LIKM2SM
    LIKDF['Log_CGEV_M2_fall']=LIKM2F
    LIKDF['Log_CGEV_M2_winter']=LIKM2W
    
    sm=0
    for i in list([LIKM2SP,LIKM2SM,LIKM2F,LIKM2W]):
        
        if np.isnan(i):
            sm=sm+np.nan_to_num(i)
        else:
            sm=sm+i
    LIKDF['Log_CGEV_M2_totsl']=sm 
    LIKDF=LIKDF.drop(columns=[0])
    
    LIK=Likelihood(EseasonT,'Like12name','Like48name',param,param1)

    Result=pd.concat([Result,INS1M1,INS1M2SP,INS1M2SM,INS1M2F,INS1M2W],axis=1)
    
    Result['RL_GEV_SP']=Rspring
    Result['RL_GEV_SM']=Rsummer
    Result['RL_GEV_F']=Rfall
    Result['RL_GEV_W']=Rwinter
    Result=pd.concat([Result,scenario2,scenario2_model2,LIKDF],axis=1)
    
    Result['Log_CGEV_4season']=LIK['Like48name'].unique().sum()
    Result['Log_CGEV_20season']=LIK['Like12name'].sum()
    Result = Result.loc[:,~Result.columns.duplicated()]
    

    Result['Log_GEV_W']=Likewinter
    Result['Log_GEV_SP']=Likespring
    Result['Log_GEV_SM']=Likesummer
    Result['Log_GEV_F']=Likefall
    Result['Log_GEV_3P']=Like3p
    return Result








def Likelihood_Model2(name,EseasonT,param,param1):
    EseasonT=EseasonT.loc[EseasonT['season']==name]
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    List
    s=0
    EseasonT=EseasonT.dropna(thresh=10)
    EseasonT=EseasonT.reset_index(inplace=False)
    EseasonT=EseasonT.drop('index', axis=1)
    Lik=0

    upbound=0
    pp=np.array([])
    Etta_D=param1[0]
    if Etta_D>1:
        Etta_D=.65

    Etta_d=param[0]
    if Etta_d>1:
        Etta_d=.65
    
    Mu_D=param1[1]
    Mu_d=param[1]
    Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)
    Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
    print('Gd'+str(Gamma_d))
    print('GD'+str(Gamma_D))
    for Delta_t in list(EseasonT['avg'].unique()):
        newdf=EseasonT.loc[EseasonT['avg']==Delta_t]
        index=EseasonT.loc[EseasonT['avg']==Delta_t].index
        List=ListC(newdf)
        newdf=newdf[List]
        newdf=newdf.to_numpy()
        newdf=newdf.reshape(-1,1)
        newdf = newdf[~np.isnan(newdf)]
        newdf=newdf.tolist()

        Lik=0
        for o in newdf:
            x=o
            print('x'+str(x))
            GEV_GEV_function=lambda j: (1/j)*(np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
            upbound1=((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D
            upbound2=(Gamma_d*x)/(Gamma_d*Mu_d-1)
            if upbound1>upbound2:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
            if upbound2>upbound1:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                            
            fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
            
            if ((fx1+fx2+fx3+fx4)*fx5)>0:
                
                fx=math.log((fx1+fx2+fx3+fx4)*fx5)
                Lik=fx+Lik
            else:
                Lik=np.nan
                break

    return Lik
         
        





# for invers CGEV consider different parameter to pass to model model2
def inverseS1_model2(name,EseasonT,param,param1):
    EseasonT=EseasonT.loc[EseasonT['season']==name]
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    Etta_D=param1[0]

    
    Etta_d=param[0]

    Mu_D=param1[1]
    Mu_d=param[1]
    Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)
    Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
    Inverse=pd.DataFrame(np.array([[0,0]]),columns=['RL_CGEV_S1M2  '+name,'RL_CGEV_S1M2  '+name+'AVG'])
    for Delta_t in list(EseasonT['avg'].unique()):
        newdf=EseasonT.loc[EseasonT['avg']==Delta_t]
        newdf1=newdf[List]
        AVG=sum(newdf1.max(axis=1)*newdf['Mean'])/newdf1.shape[0]
        
        GEV_GEV_function=lambda z,j: (np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*z))**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*z))**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
                            
        fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
        
        upy=(((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D)
        upz=(Gamma_d*Mu_d-1)/(Gamma_d)
        print('upy: '+str(upy))
        print('upz: '+str(upz))
        t=upz/5
        
        if Etta_d>0:
            t=upz+1
            if t<0:
                t=t*(-1)
        
        
        result=0
        n=0
        s=0
        while abs(0.91667-result)>0.005:
            cond1=0

            


            if (Etta_d<0) & (Etta_D<0):
                #both negative
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
            elif (Etta_d>0) & (Etta_D>0):
                #both positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)

            elif (Etta_d>0) & (Etta_D<0):
                # Etta_D is negative and Etta_d is positive    
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                            
            elif (Etta_d<0) & (Etta_D>0):
                # Etta_d is negative and Etta_D is positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                    
            print('**'+str(fx))        
            if fx<0.0001:
                
                s=s+1
                print(s)
                if s>5:
                    cond1=3
                    break
            result1=fx

            if (0.91667-result1)*2<1.8:
                if abs(result1-result)<0.005:
                    if abs(0.91667-result)>.1:
                        
                        cond1=2
                        break
                result=result1
                t=(t+(0.91667-result)*2)
                
            else:
                t=upz/(4*s)
                if Etta_d>0:
                    t=upz+2*s
                    if t<0:
                        t=t*-1
                t=(t+(0.91667-result)*2)
                if s>5:
                    cond1=1
                    break

            
            print(t)
            print((0.91667-result))
            n=n+1
            if n>45:
                break
            
        if (n>45) or (cond1==1) or (cond1==2) or (cond1==3):
            t=t*Delta_t
            Inverse['RL_CGEV_S1M2'+'  '+list(newdf['season'].unique())[0]]=str(t)+'**'+str(result)+'**'+str(cond1)
        else:
            t=t*Delta_t
            Inverse['RL_CGEV_S1M2'+'  '+list(newdf['season'].unique())[0]]=t
            
            
        
        Inverse[list('RL_CGEV_S1M2'+'  '+newdf['season'].unique())[0]+'AVG']=AVG


    return Inverse
        
       
        
            
            

# for invers CGEV consider different parameter to pass to model model1
def inverseS1(EseasonT,param,param1):
    
    
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    Etta_D=param1[0]

    s=0
    Etta_d=param[0]

    Mu_D=param1[1]
    Mu_d=param[1]
    Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)
    Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
    Inverse=pd.DataFrame([np.array([])])
    for Delta_t in list(EseasonT['avg'].unique()):
        newdf=EseasonT.loc[EseasonT['avg']==Delta_t]
        newdf1=newdf[List]
        AVG=sum(newdf1.max(axis=1)*newdf['Mean'])/newdf1.shape[0]
        
        GEV_GEV_function=lambda z,j: (np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*z))**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*z))**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
                            
        fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
          
        upy=(((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D)
        upz=(Gamma_d*Mu_d-1)/(Gamma_d)
        t=upz/5 
        if Etta_d>0:
            t=upz+3
            if t<0:
                t=t*-1
        result=0
        n=0
        
        while abs(0.91667-result)>0.005:
            cond1=0

            


            if (Etta_d<0) & (Etta_D<0):
                #both negative
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
            elif (Etta_d>0) & (Etta_D>0):
                #both positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)

            elif (Etta_d>0) & (Etta_D<0):
                # Etta_D is negative and Etta_d is positive    
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                            
            elif (Etta_d<0) & (Etta_D>0):
                # Etta_d is negative and Etta_D is positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                    
                    
            if fx<0.0001:
                s=s+1
                if s>5:
                    cond1=3
                    break
            result1=fx

            if (0.91667-result1)*2<1.8:
                if abs(result1-result)<0.005:
                    if abs(0.91667-result)>.1:
                        
                        cond1=2
                        break
                result=result1
                t=(t+(0.91667-result)*2)
                
            else:
                t=upz/(4*s)
                if Etta_d>0:
                    t=upz+3
                if t<0:
                    t=t*-1
                t=(t+(0.91667-result)*2)
                if s>5:
                    cond1=1
                    break
                
            
            print(t)
            print((0.91667-result))
            n=n+1
            if n>45:
                break
            
        if (n>45) or (cond1==1) or (cond1==2) or (cond1==3):
            t=t*Delta_t
            Inverse['RL_CGEV_S1M1 '+list(newdf['season'].unique())[0]]=str(t)+'**'+str(result)+'**'+str(cond1)
        else:
            t=t*Delta_t
            Inverse['RL_CGEV_S1M1 '+list(newdf['season'].unique())[0]]=t
    
    
        Inverse['RL_CGEV_S1M1 '+list(newdf['season'].unique())[0]+'AVG']=AVG


    return Inverse
        
        
        
            
            


def sce2(EseasonT,seasonT):
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    List1=list(seasonT.columns)
    List1.remove('Year')
    List1.remove('season')
    List1.remove('num')
    List1.remove('Mean')
    
    scenario2=pd.DataFrame(np.array([[0,0,0,0]]),columns=['springmax','summermax','fallmax','wintermax'])
    listSeason=pd.Series(EseasonT['season'].unique())
    listSeason=list(listSeason.dropna())
    for i in listSeason:
        df=EseasonT.loc[EseasonT['season']==i]
        df1=seasonT.loc[seasonT['season']==i]
        
        listyear=df['Year'].unique()
        listyear1=df1['Year'].unique()
        df=df.loc[df['Year']==listyear.max()]
        df1=df1.loc[df1['Year']==listyear1.max()]
        index=df.loc[df['Year']==listyear.max()].index[0]
        index1=df1.loc[df1['Year']==listyear1.max()].index[0]
        df2=df[List]
        max2=np.array(df2.max(axis=1)*df['Mean'])
        scenario2[list(df['season'].unique())[0]+'max']=max2
        EseasonT=EseasonT.drop([index])
        seasonT=seasonT.drop([index1])
        EseasonT=EseasonT.dropna(thresh=10)
    return EseasonT,seasonT ,scenario2
    
    











#scneario 2 model 1




def inverseS2(EseasonT,seasonT,scenario2):
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    

        
    param1=parameterSeasonal(seasonT)
    llh,param=LikelihoodGEV(EseasonT[List])
    Etta_D=param1[0]

    s=0
    Etta_d=param[0]

    Mu_D=param1[1]
    Mu_d=param[1]
    Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)
    print(Etta_d)
    print(Mu_d)
    Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
    
    for Delta_t in list(EseasonT['avg'].unique()):
        newdf=EseasonT.loc[EseasonT['avg']==Delta_t]
        newdf1=newdf[List]
        AVG=sum(newdf1.max(axis=1)*newdf['Mean'])/newdf1.shape[0]
        
        GEV_GEV_function=lambda z,j: (np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*z))**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*z))**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
                            
        fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
          
        upy=(((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D)
        upz=(Gamma_d*Mu_d-1)/(Gamma_d)
        t=upz/5
        if Etta_d>0:
            t=upz+3
            if t<0:
                t=t*-1
        result=0
        n=0
        
        while abs(0.91667-result)>0.005:
            cond1=0

            


            if (Etta_d<0) & (Etta_D<0):
                #both negative
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
            elif (Etta_d>0) & (Etta_D>0):
                #both positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)

            elif (Etta_d>0) & (Etta_D<0):
                # Etta_D is negative and Etta_d is positive    
                fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :upz,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                            
            elif (Etta_d<0) & (Etta_D>0):
                # Etta_d is negative and Etta_D is positive
                fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :-np.inf,lambda j:t)[0]*fx5
                if np.isnan(fx):
                    fx=np.nan_to_num(fx)
                    
                    
            if fx<0.0001:
                s=s+1
                if s>5:
                    cond1=3
                    break
            result1=fx

            if (0.91667-result1)*2<1.8:
                if abs(result1-result)<0.005:
                    if abs(0.91667-result)>.1:
                        
                        cond1=2
                        break
                result=result1
                
            else:
                if s>5:
                    
                    cond1=1
                    break
                
            t=(t+(0.91667-result)*2)
            print(t)
            print((0.91667-result))
            n=n+1
            if n>45:
                break
            
        if (n>45) or (cond1==1) or (cond1==2) or (cond1==3):
            t=t*Delta_t
            scenario2['RL_CGEV_M1S2 '+list(newdf['season'].unique())[0]]=str(t)+'**'+str(result)+'**'+str(cond1)
        else:
            t=t*Delta_t
            scenario2['RL_CGEV_M1S2 '+list(newdf['season'].unique())[0]]=t

        
        
    return scenario2
        
        
        

        
  











        
        
def inverseS2_model2(EseasonT,seasonT,scenario2):
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    

    seasonU=seasonT.season.unique()    
    for i in seasonU:
        print('*****'+str(i))
        param1=parameterSeasonal(seasonT.loc[seasonT['season']==i])
        EseasonT1=EseasonT.loc[EseasonT['season']==i]
        llh,param=LikelihoodGEV(EseasonT1[List])
        Etta_D=param1[0]
        

        print('E_D'+str(Etta_D))
        s=0
        Etta_d=param[0]
 
        print('E_d'+str(Etta_d))
        Mu_D=param1[1]
        Mu_d=param[1]
        Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)

        Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
        
        for Delta_t in list(EseasonT1['avg'].unique()):
            print('Del'+str(Delta_t))
            newdf=EseasonT1.loc[EseasonT1['avg']==Delta_t]
            newdf1=newdf[List]
            AVG=sum(newdf1.max(axis=1)*newdf['Mean'])/newdf1.shape[0]
            
            GEV_GEV_function=lambda z,j: (np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*z))**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*z))**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
                                
            fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
              
            upy=(((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D)
            upz=(Gamma_d*Mu_d-1)/(Gamma_d)
            print('upy'+str(upy))
            print('upz'+str(upz))
            t=upz/5
            if Etta_d>0:
                t=upz+3
                if t<0:
                    t=t*-1
            result=0
            n=0
        
            while abs(0.91667-result)>0.005:
                cond1=0

                
    

                if (Etta_d<0) & (Etta_D<0):
                    #both negative
                    fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :-np.inf,lambda j:t)[0]*fx5
                    if np.isnan(fx):
                        fx=np.nan_to_num(fx)
                elif (Etta_d>0) & (Etta_D>0):
                    #both positive
                    fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :upz,lambda j:t)[0]*fx5
                    if np.isnan(fx):
                        fx=np.nan_to_num(fx)

                elif (Etta_d>0) & (Etta_D<0):
                    # Etta_D is negative and Etta_d is positive    
                    fx=integrate.dblquad(GEV_GEV_function,-np.inf,upy, lambda j :upz,lambda j:t)[0]*fx5
                    if np.isnan(fx):
                        fx=np.nan_to_num(fx)
                                
                elif (Etta_d<0) & (Etta_D>0):
                    # Etta_d is negative and Etta_D is positive
                    fx=integrate.dblquad(GEV_GEV_function,upy,np.inf, lambda j :-np.inf,lambda j:t)[0]*fx5
                    if np.isnan(fx):
                        fx=np.nan_to_num(fx)
                        
                print('fx='+str(fx))       
                if fx<0.0001:
                    s=s+1
                    if s>5:
                        cond1=3
                        break
                result1=fx

                if (0.91667-result1)*2<1.8:
                    if abs(result1-result)<0.005:
                        if abs(0.91667-result)>.1:
                            
                            cond1=2
                            break
                    result=result1
                    
                else:
                    if s>5:
                        
                        cond1=1
                        break
                    
                t=(t+(0.91667-result)*2)
                print(t)
                print((0.91667-result))
                n=n+1
                if n>45:
                    break
            
            if (n>45) or (cond1==1) or (cond1==2) or (cond1==3):
                t=t*Delta_t
                scenario2['RL_CGEV_M2S2 '+list(newdf['season'].unique())[0]]=str(t)+'**'+str(result)+'**'+str(cond1)
            else:
                t=t*Delta_t
                scenario2['RL_CGEV_M2S2 '+list(newdf['season'].unique())[0]]=t

        
        
    return scenario2
            




















def Likelihood(EseasonT,Like12name,Like48name,param,param1):
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    List
    EseasonT[Like48name]=0 
    EseasonT[Like12name]=0
    EseasonT=EseasonT.dropna(thresh=10)
    EseasonT=EseasonT.reset_index(inplace=False)
    EseasonT=EseasonT.drop('index', axis=1)
    Lik=0
    
    upbound=0
    pp=np.array([])
    Etta_D=param1[0]
    if Etta_D>1:
        Etta_D=.65

    Etta_d=param[0]
    if Etta_d>1:
        Etta_d=.65
    Mu_D=param1[1]
    Mu_d=param[1]
    Gamma_D = (gamma(1-Etta_D)-1)/(1-Mu_D)
    Gamma_d = (gamma(1-Etta_d)-1)/(1-Mu_d)
    for Delta_t in EseasonT['Mean']:
        newdf=EseasonT.loc[EseasonT['Mean']==Delta_t]
        index=EseasonT.loc[EseasonT['Mean']==Delta_t].index[0]
        
        List=ListC(newdf)
        newdf=newdf[List]
        newdf=newdf.to_numpy()
        newdf=newdf.reshape(-1,1)
        newdf = newdf[~np.isnan(newdf)]
        newdf=newdf.tolist()
        Lik=0
        
        for o in newdf:
            x=o
            GEV_GEV_function=lambda j: (1/j)*(np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
            upbound1=((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D
            upbound2=(Gamma_d*x)/(Gamma_d*Mu_d-1)
            if upbound1>upbound2:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
            if upbound2>upbound1:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                            
            fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
            pp=np.append(pp,np.array([fx1]))
            
            fx=np.log((fx1+fx2+fx3+fx4)*fx5)
            if np.isnan(fx):
                fx=np.nan_to_num(fx)
            Lik=fx+Lik
            
        EseasonT[Like12name].iloc[index]=Lik
    
    for Delta_t in list(EseasonT['avg'].unique()):
        newdf=EseasonT.loc[EseasonT['avg']==Delta_t]
        index=EseasonT.loc[EseasonT['avg']==Delta_t].index
        List=ListC(newdf)
        newdf=newdf[List]
        newdf=newdf.to_numpy()
        newdf=newdf.reshape(-1,1)
        newdf = newdf[~np.isnan(newdf)]
        newdf=newdf.tolist()
        Lik=0
        for o in newdf:
            x=o
            GEV_GEV_function=lambda j: (1/j)*(np.exp(-(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**((-1)/Etta_d))*np.exp(-(1-(Mu_D*Gamma_D)+(Gamma_D*j)/Delta_t)**((-1)/Etta_D)))*(1-(Mu_d*Gamma_d)+(Gamma_d*x)/j)**(-1-(1/Etta_d))*(1-Mu_D*Gamma_D+Gamma_D*j/Delta_t)**(-1-(1/Etta_D))
            upbound1=((Gamma_D*Mu_D-1)*Delta_t)/Gamma_D
            upbound2=(Gamma_d*x)/(Gamma_d*Mu_d-1)
            if upbound1>upbound2:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound1,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound2)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound2,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
            if upbound2>upbound1:
                if upbound1*upbound2<0:
                    fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                    if np.isnan(fx1):
                        fx1=np.nan_to_num(fx1)
                    fx2=integrate.quad(GEV_GEV_function,upbound1,-0.001)[0]
                    if np.isnan(fx2):
                        fx2=np.nan_to_num(fx2)
                    fx3=integrate.quad(GEV_GEV_function,0.001,upbound2)[0]
                    if np.isnan(fx3):
                        fx3=np.nan_to_num(fx3)
                    fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                    if np.isnan(fx4):
                        fx4=np.nan_to_num(fx4)
                elif upbound1*upbound2>0:
                    if upbound1>0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,-0.001)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,0.001,upbound1)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,upbound2,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                    elif upbound1<0:
                        fx1=integrate.quad(GEV_GEV_function,-np.inf,upbound1)[0]
                        if np.isnan(fx1):
                            fx1=np.nan_to_num(fx1)
                        fx2=integrate.quad(GEV_GEV_function,upbound1,upbound2)[0]
                        if np.isnan(fx2):
                            fx2=np.nan_to_num(fx2)
                        fx3=integrate.quad(GEV_GEV_function,upbound2,-0.001)[0]
                        if np.isnan(fx3):
                            fx3=np.nan_to_num(fx3)
                        fx4=integrate.quad(GEV_GEV_function,0.001,np.inf)[0]
                        if np.isnan(fx4):
                            fx4=np.nan_to_num(fx4)
                            
            fx5 = ((Gamma_d*Gamma_D)/(Delta_t*Etta_d*Etta_D))
            
            
            fx=np.log((fx1+fx2+fx3+fx4)*fx5)
            if np.isnan(fx):
                fx=np.nan_to_num(fx)
            Lik=fx+Lik

        for t in list(index):
            EseasonT[Like48name].iloc[t]=Lik 
    return EseasonT
         
        


# fitting and get parameter for model 1 and 2
def parameters_Eseason(Meantype,error1,error2,error3,error4,df):
    
    df1=df
    spr=season('spring',3,4,5,df1,Meantype).drop_duplicates(inplace=False)
    summ=season('summer',6,7,8,df1,Meantype).drop_duplicates(inplace=False)
    fall=season('fall',9,10,11,df1,Meantype).drop_duplicates(inplace=False)
    winter=season1('winter',df,Meantype)
    seasonT=pd.DataFrame([])
    seasonT=seasonT.append([spr,summ,fall,winter])
    #Fitting for model 1
    # Fitting for parameter 
    dfFit=pd.DataFrame([])
    dfFit=dfFit.append([df[error1],df[error2],df[error3],df[error4]])
    y=dfFit.to_numpy()
    y=y.reshape(-1,1)
    dfFit=pd.DataFrame(y)
    dfFit=dfFit[dfFit>0]
    dfFit=dfFit.dropna()
    y=dfFit.to_numpy()
    dist = getattr(scipy.stats, 'genextreme')
    param = dist.fit(y)
    
    
    #Fitting for parameter 
    
    df11=seasonT.reset_index(inplace = False)
    dfFit=pd.DataFrame([])
    dfFit=dfFit.append([df11['e1'] , df11['e2'] ,df11['e3']])
    y=dfFit.to_numpy()
    y=y.reshape(-1,1)
    dfFit=pd.DataFrame(y)
    dfFit=dfFit[dfFit>0]
    dfFit=dfFit.dropna()
    y=dfFit.to_numpy()
    
    dist = getattr(scipy.stats, 'genextreme')
    param1 = dist.fit(y)
    EseasonT=pd.DataFrame([])
    Espr=Eseason('spring',3,4,5,df1,Meantype,error1,error2,error3,error4).drop_duplicates(inplace=False)
    Esumm=Eseason('summer',6,7,8,df1,Meantype,error1,error2,error3,error4).drop_duplicates(inplace=False)
    Efall=Eseason('fall',9,10,11,df1,Meantype,error1,error2,error3,error4).drop_duplicates(inplace=False)
    Ewinter=Eseason1('winter',df,Meantype,error1,error2,error3,error4)
    if Espr.shape[0]>0:
        
        List=ListC(Espr)
        Likespring,param_primespring=LikelihoodGEV(Espr[List])
        Rspring=returnvalue(param_primespring,Espr.dropna(thresh=10).avg[0])
        if str(type(Rspring))!="<class 'numpy.float64'>":
            Rspring=Rspring.drop_duplicates(inplace=False)
        EseasonT=EseasonT.append([Espr])
    else:
        Rspring=np.nan
        Likespring=np.nan
        param_primespring=np.nan
        
    if Esumm.shape[0]>0:
            
        List=ListC(Esumm)
        Likesummer,param_primesummer=LikelihoodGEV(Esumm[List])
        Rsummer=returnvalue(param_primesummer,Esumm.dropna(thresh=10).avg[0])
        if str(type(Rsummer))!="<class 'numpy.float64'>":
            
            Rsummer=Rsummer.drop_duplicates(inplace=False)
        EseasonT=EseasonT.append([Esumm])
    else:
        Rsummer=np.nan
        Likesummer=np.nan
        param_primesummer=np.nan
    if Efall.shape[0]:
            
        List=ListC(Efall)
        Likefall,param_primefall=LikelihoodGEV(Efall[List])
        Rfall=returnvalue(param_primefall,Efall.dropna(thresh=10).avg[0])
        if str(type(Rfall))!="<class 'numpy.float64'>":
            
            Rfall=Rfall.drop_duplicates(inplace=False)
        EseasonT=EseasonT.append([Efall])
    else:
        Rfall=np.nan
        Likefall=np.nan
        param_primefall=np.nan
    if Ewinter.shape[0]>0:
        
        List=ListC(Ewinter)
        Likewinter,param_primewinter=LikelihoodGEV(Ewinter[List])
        Rwinter=returnvalue(param_primewinter,Ewinter.dropna(thresh=10).avg[0])
        if str(type(Rwinter))!="<class 'numpy.float64'>":
            Rwinter=Rwinter.drop_duplicates(inplace=False)
        EseasonT=EseasonT.append([Ewinter])
    else:
        Rwinter=np.nan
        Likewinter=np.nan
        param_primewinter=np.nan
        
    if spr.shape[0]>0:
        
        param1_primespring=parameterSeasonal(spr)
    else:
        param1_primespring=np.nan
    if summ.shape[0]>0:
        
        param1_primesummer=parameterSeasonal(summ)
    else:
        param1_primesummer=np.nan
    if fall.shape[0]>0:
        
        param1_primefall=parameterSeasonal(fall)
    else:
        param1_primefall=np.nan
    if winter.shape[0]>0:
        
        param1_primewinter=parameterSeasonal(winter)
    else:
        param1_primewinter=np.nan
    
    
    
    
    EseasonT=EseasonT.reset_index(inplace=False)
    EseasonT=EseasonT.drop('index', axis=1)
    seasonT=seasonT.reset_index(inplace=False)
    seasonT=seasonT.drop('index', axis=1)
    EseasonT=EseasonT.dropna(thresh=10)
    Like3p,ppp=LikelihoodGEV(EseasonT[List])
    return param,param1,seasonT,EseasonT,Likespring,Likesummer,Likefall,Likewinter,Like3p,param_primespring,param1_primespring,param_primesummer,param1_primesummer,param_primefall,param1_primefall,param_primewinter,param1_primewinter,Rspring,Rsummer,Rfall,Rwinter



























def season1(name,df1,Meantype):
    cond=1
    for i in range(3,12):
        df1=df1[df1.Month != i]
    df1['Month'] =df1['Month'].map({12 : 0, 1 :1 , 2:2})
    yearU=df.Year.unique()
    yearU1=yearU-1
    dfT=pd.DataFrame([])
    for i , j in zip(yearU, yearU1):
        
        df21=df1.loc[(df1['Year']== i) & (df1['Month']!=0)]
        
        df21=df21.append(df1.loc[(df['Year']== j)& (df1['Month']==0)])
        dfF=pd.DataFrame(columns=['Year','season','Mean','STD','e1','e2','e3'])
        dfF=dfF.append([0,0,0,0,0,0,0])
        df21=df21[df21[Meantype]>0]
        if df21.Year.count() ==3:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['STD']=df21[Meantype].std()
            cont=range(df21.Year.count())
            dfF['e1']=(df21[Meantype].iloc[cont[0]])/(df21[Meantype].mean())
            dfF['e2']=(df21[Meantype].iloc[cont[1]])/(df21[Meantype].mean())
            dfF['e3']=(df21[Meantype].iloc[cont[2]])/(df21[Meantype].mean())
        elif df21.Year.count() ==2:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['STD']=df21[Meantype].std()
            cont=range(df21.Year.count())
            dfF['e1']=(df21[Meantype].iloc[cont[0]])/(df21[Meantype].mean())
            dfF['e2']=(df21[Meantype].iloc[cont[1]])/(df21[Meantype].mean())
            dfF['e3']=None
        elif df21.Year.count() ==1:
            cond=1
        else:
            continue
        if cond==0:
            dfT=dfT.append(dfF)
            
        
    dfT=dfT.drop_duplicates(inplace=False)
    return dfT















def season (name,a,b,c,df1,Meantype):
    dfT=pd.DataFrame([])
    df2=df1.loc[(df1['Month']== a) | (df1['Month']==(b)) | (df1['Month']==(c)) ]
    yearU=df2.Year.unique()
    for i in yearU:    
        df21=df2.loc[(df2['Year']== i)]
        dfF=pd.DataFrame(columns=['Year','season','Mean','STD','e1','e2','e3','num'])
        dfF=dfF.append([0,0,0,0,0,0,0])
        
        df21=df21[df21[Meantype]>0]
        if df21.Year.count()==3:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['STD']=df21[Meantype].std()
            cont=range(df21.Year.count())
            dfF['e1']=(df21[Meantype].iloc[cont[0]])/(df21[Meantype].mean())
            dfF['e2']=(df21[Meantype].iloc[cont[1]])/(df21[Meantype].mean())
            dfF['e3']=(df21[Meantype].iloc[cont[2]])/(df21[Meantype].mean())
            
        elif df21.Year.count() ==2:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['STD']=df21[Meantype].std()
            cont=range(df21.Year.count())
            dfF['e1']=(df21[Meantype].iloc[cont[0]])/(df21[Meantype].mean())
            dfF['e2']=(df21[Meantype].iloc[cont[1]])/(df21[Meantype].mean())
            dfF['e3']=None
            
        elif df21.Year.count() ==1:
            cond=1
        else:
            continue
        if cond==0:
            dfT=dfT.append(dfF)
            
        
    dfT=dfT.drop_duplicates(inplace=False)
    return dfT






def Eseason (name,a,b,c,df1,Meantype,error1,error2,error3,error4):
    dfT=pd.DataFrame([])
    dfF=pd.DataFrame([])
    df2=df1.loc[(df1['Month']== a) | (df1['Month']==(b)) | (df1['Month']==(c)) ]
    
    yearU=df2.Year.unique()
    for i in yearU:    
        df21=df2.loc[(df2['Year']== i)]
        df21=df21[df21[Meantype]>0]
        
        df31=df21[[error1,error2,error3,error4]]
        df31=df31[df31>0]
        
        df31=df31.to_numpy().reshape(1,-1)
        df31= df31[~np.isnan(df31)]
        dfF=pd.DataFrame([df31])

        if df21.Year.count() ==3:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['num']=df21.Year.count()


        elif df21.Year.count() ==2:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['num']=df21.Year.count()


        elif df21.Year.count() ==1:
            cond=1
            dfF['num']=1
        else:
            continue
        if cond==0:
            dfT=dfT.append(dfF)
        if df21.Year.count() >1:
            sum1=dfT['num']*dfT['Mean']
            dfT['avg']=sum1.sum()/(dfT['num'].sum())
        

    
    
    return dfT



def Eseason1(name,df1,Meantype,error1,error2,error3,error4):
    cond=1
    for i in range(3,12):
        df1=df1[df1.Month != i]
    df1['Month'] =df1['Month'].map({12 : 0, 1 :1 , 2:2})
    
    yearU=df.Year.unique()
    yearU1=yearU-1
    dfT=pd.DataFrame([])
    for i , j in zip(yearU, yearU1):
        
        df21=df1.loc[(df1['Year']== i) & (df1['Month']!=0)]
        
        
        df21=df21.append(df1.loc[(df['Year']== j)& (df1['Month']==0)])
        df21=df21[df21[Meantype]>0]
        df31=df21[[error1,error2,error3,error4]]
        df31=df31[df31>0]
        
        df31=df31.to_numpy().reshape(1,-1)
        df31= df31[~np.isnan(df31)]
        dfF=pd.DataFrame([df31])
        
        
        if df21.Year.count() ==3:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['num']=df21.Year.count()

        elif df21.Year.count() ==2:
            cond=0
            dfF['Year']=i
            dfF['season']=str(name)
            dfF['Mean']=df21[Meantype].mean()
            dfF['num']=df21.Year.count()
  
        elif df21.Year.count() ==1:
            cond=1
            dfF['num']=df21.Year.count()
        else:
            continue
        if cond==0:
            dfT=dfT.append(dfF)
        
        if df21.Year.count() >1:
            sum1=dfT['num']*dfT['Mean']
            dfT['avg']=sum1.sum()/(dfT['num'].sum())
        
    
       
    
    return dfT





def ListC(EseasonT):
    List=list(EseasonT.columns)
    List.remove('Year')
    List.remove('season')
    List.remove('num')
    List.remove('Mean')
    List.remove('avg')
    return List
# function for fitting parameter and calculate likelihood
def LikelihoodGEV(dfFit):
    LLH=0
    y=dfFit.to_numpy()
    y=y.reshape(-1,1)
    dfFit=pd.DataFrame(y)
    dfFit=dfFit[dfFit>0]
    dfFit=dfFit.dropna()
    y=dfFit.to_numpy()
    if y.shape[0]>2:
        dist = getattr(scipy.stats, 'genextreme')
        param_prime2 = dist.fit(y)
        LLH = dist.logpdf(y,*param_prime2).sum()
        return LLH,param_prime2
    else:
        return np.nan,np.nan
    

# function for fitting parameter 
def parameterSeasonal(dfseason):
    df11=dfseason.reset_index(inplace = False)
    dfFit=pd.DataFrame([])
    dfFit=dfFit.append([df11['e1'] , df11['e2'] ,df11['e3']])
    y=dfFit.to_numpy()
    y=y.reshape(-1,1)
    dfFit=pd.DataFrame(y)
    dfFit=dfFit[dfFit>0]
    dfFit=dfFit.dropna()
    y=dfFit.to_numpy()
    if y.shape[0]>2:
        dist = getattr(scipy.stats, 'genextreme')
        param_prime2 = dist.fit(y)
        LLH = dist.logpdf(y,*param_prime2).sum()
        return param_prime2
    else:
        return np.nan
    

def returnvalue(prameter,delta_avg):
    
    pr=prameter
    if np.nan_to_num(pr[0])==0:
        
        return 'notdefine'
    else:
        dist = getattr(scipy.stats, 'genextreme')
        ret = (pr[1]-(pr[2]/pr[0])*(1-(-np.log(.916667))**(-pr[0])))*delta_avg
    return ret

    
